# TUNNIG, FIT AND EXPORT

### Búsqueda de hiperparámetros, ajuste del modelo y exportacion a Kaggle

In [ ]:
# Importaremos el modelo y prueba que nos dio mejor resultado (usamos un mean encoding sin seed)
Modelo = pd.read_csv("../input/data22222/Modelo (1).csv")
Prueba = pd.read_csv("../input/data22222/Prueba (1).csv")
X=Modelo.iloc[:,:-1]
y=Modelo.iloc[:,-1].to_frame()
Xp=Prueba.iloc[:,:-1]

In [ ]:
#Creamos un conjunto de hiperparametros a probar,estas configuraciones serán probadas y los resultados serán guardados en un csv nombrado partial_results.csv:

params={'alpha': [0],
        'colsample_bylevel': [1,2,3],
        'colsample_bynode': [1],
        'colsample_bytree': [0.3,0.35,0.4],
        'eval_metric': 'auc',
        'gamma': [0.8,0.9,1],
        'lambda': [1],
        'learning_rate': [0.0008],
        'max_delta_step': [0],
        'max_depth': [19], 
        'min_child_weight': [40],
        'n_estimators': [10000],
        'scale_pos_weight': [32.26]}    #Usado para combatir el desbalanceo de datos, el valor es la proporción de 0 respecto a 1 en Target


# Creamos un csv denominado partial_results.csv:
def create_partial_results_csv():
    df = pd.DataFrame(columns=['alpha', 'colsample_bylevel', 'colsample_bynode', 'colsample_bytree', 'eval_metric', 'gamma', 'lambda', 'learning_rate', 'max_delta_step', 'max_depth', 'min_child_weight', 'n_estimators', 'scale_pos_weight', 'best_score', 'minutes', 'seconds', 'params'])
    df.to_csv('partial_results.csv', index=False)
create_partial_results_csv()

#Creamos un kfold estratificado con 5 grupos que seran usados para probar el perfomance del modelo en un grid search:

kfold = StratifiedKFold(n_splits=5)  

# Definimos un grid search:
def grid_search(params, reg=XGBClassifier(booster='gbtree', objective='binary:logistic',missing=-999.0,random_state=2,n_jobs=-1,tree_method='gpu_hist',eval_metric='auc')):
    grid_reg = GridSearchCV(reg, params, scoring='roc_auc', cv=kfold)
    grid_reg.fit(X, y.values.ravel())
    global best_params 
    best_params = grid_reg.best_params_
    print("Best params:", best_params)
    best_score = grid_reg.best_score_
    print("Best score:", best_score)
    return best_params, best_score

# Creamos una lista que guardara cada combinación de hiperparametros con el respectivo score(auc) que obtienen:
def get_result_list(params):
    start = time.time()
    par, sco = grid_search(params)
    end = time.time()
    t = end - start
    m = int(t//60)
    s = round(t%60,2)
    l = list(par.values())
    l.append(sco)
    l.append(m)
    l.append(s)
    l.append(par)
    return l;

#Escribimos en el csv cada objeto de la lista creada (combinación de parámetros con score)
from csv import writer
def write_list_in_csv(l, csv_name='partial_results.csv'):
    with open(csv_name, 'a') as f_object:
        writer_object = writer(f_object)
        writer_object.writerow(l)
        f_object.close()

#Definimos una función que devuelve los parametros de los csv creados por índice
def get_params_from_csv(index, csv_name='partial_results.csv'):
    # Devuelve un diccionario con los parametros de una fila de partial_results.csv
    dfpr = pd.read_csv(csv_name)
    foo = dfpr['params'][index]
    import ast
    return ast.literal_eval(foo)

# Devuelve un csv listo para mandar a kaggle usando los parametros de params (dict)
def export_send_csv(params, name='envioJ.csv'):
    mod=XGBClassifier(booster='gbtree', objective='binary:logistic',missing=-999.0,random_state=3,n_jobs=-1,tree_method='gpu_hist',eval_metric='auc')
    mod.set_params(**params)

    start = time.time()
    mod.fit(X,y)
    end = time.time()
    t = end - start
    print(f'Demoro: {int(t//60)} minutos y {round(t%60,2)} segundos')
    prediccion_final=mod.predict_proba(Xp)[:,1]
    kek=pd.DataFrame(prediccion_final)
    kek.rename(columns={0:'TARGET'}, inplace=True)
    toconcat=[go[go["NUMPERIODO"]==202204][["nro_telefono_hash"]].reset_index(drop=True),kek.reset_index(drop=True)]
    ya=pd.concat(toconcat,axis=1,ignore_index=True)
    ya.rename(columns={0:"nro_telefono_hash",1:'TARGET'}, inplace=True)
    ya.to_csv(name, index=False)



write_list_in_csv(get_result_list(params))
da = pd.read_csv('partial_results.csv')
# Observamos las combinaciones de hiperparametros con mejores puntajes (revisamos el indice de 5 los mejores modelos)
da.sort_values(by='best_score', ascending=False).head(20)
#U sando esos indices obtenemos sus parametros
get_params_from_csv(0, csv_name='partial_results.csv'):

# Crea un csv listo para mandar a kaggle con alguna de las filas del partial_results.csv
export_send_csv(get_params_from_csv(32), 'envioJV32.csv')